In [1]:
# Import library
import requests
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import base64
from requests.exceptions import ReadTimeout
import csv
import datetime
import traceback
import sys

In [16]:
# IDs

#1.1
#client_id = "e4b7435210ed4f8598f14c2eb13e5412"
#client_secret = "82169d73e683483db02b492cc25231cd"

#1.2
#client_id = "65902791925549cb8d1a76764f08d9c9"
#client_secret = "e29852f240ff4f4aa7cc41ae4fd294b9"

#1.3
#client_id = "8fe545dfe43f42359afd13167713a679"
#client_secret = "e2a8ec2e534140568777741540e2d2f5"

#1.4
client_id = "305564a058c64bb5aca31ea35e68fc2a"
client_secret = "435171bb8e224583820ba8054b14db1c"

#1.5
#client_id = "0400ee04258f41ffa9cdbfb242470f0e"
#client_secret = "713383fe08c64795a283c0f859f24a33"

#2.1
#client_id = "75a7c1edf8784fd3862d4e0930894742"
#client_secret = "789f1005c0254a999d6b1ef53506d153"

#2.2
#client_id = "21be1429712f4c4fb60afef65b52e51c"
#client_secret = "860cddd1840e46c4a358e4a17af5f01a"

#2.3
#client_id = "d042b85f700b4e80960e99e5a7283603"
#client_secret = "98cea237ef4b4011a3f4c5154cef3923"

#2.4
#client_id = "dee5e2ce0b1b4d7982e0a8caced3da1f"
#client_secret = "4a6a815d6320416ca0ff0a5a68a1f346"

#2.5
#client_id = "02a0a895986148bd838da1935dd5c864"
#client_secret = "b04ac15e5a8f4df7ad6bb5dd01a14d33"

#3.1
#client_id = "eadffed752674760afd13f0804c84900"
#client_secret = "81aeffa80c7e4f309d58bd83d41dfb10"

#3.2
#client_id = "86b548dfbb684998ac97dc14309f3727"
#client_secret = "63cd3c4c2c47497bb21654c745fa4a8e"

#3.3
#client_id = "ceeb073b19184749ae69a6d1557c4e31"
#client_secret = "08cb1a9f4d6f414ca844eafa0b9913e5"

#3.4
#client_id = "1555a0336d904ca7a4f35e2c2e6597d8"
#client_secret = "3528d103d36349099c1cba15a3fd7729"

#3.5
#client_id = "1dd05312d3464db2b242cc5f7864e2b7"
#client_secret = "dd21ed8eda2e4fceb0428c83235fcf57"


In [17]:
headers = {}
client_creds = f"{client_id}:{client_secret}"
client_creds_base64 = base64.b64encode(client_creds.encode())
token_url = 'https://accounts.spotify.com/api/token'
token_data = {"grant_type": "client_credentials"}
token_headers = {"Authorization":f"Basic {client_creds_base64.decode()}" }

def get_new_token():
    r = requests.post(token_url, headers=token_headers, data=token_data)
    valid_request = r.status_code in range(200, 299)

    if valid_request:
        r = r.json()
        access_token = r['access_token']
        expires_in = r["expires_in"]    # in second
        # print(access_token, expires_in)

        now = datetime.datetime.now()
        expires = now + datetime.timedelta(seconds=expires_in)
        # did_expire = expires < now

        # Construct Spotify API query
        headers = {
            'Authorization': 'Bearer '+ access_token,
            'Content-type': 'application/json',
        }
    return expires, headers

expires, headers = get_new_token()

In [132]:
track_url = 'https://api.spotify.com/v1/recommendations/available-genre-seeds'

# Query Spotify API
try:
    if expires < datetime.datetime.now(): 
        print("Get new Token")
        expires, headers = get_new_token()

    response = requests.get(track_url, headers=headers)
    if response.status_code == 200:
        genres = response.json()
        print(json.dumps(genres, indent=4))

        genres_to_remove = [#"acoustic", "afrobeat", "alt-rock", "alternative", "ambient", "anime", "black-metal", "bluegrass", "blues", "bossanova",
                            #"brazil","breakbeat", "british", "cantopop", "chicago-house", "children", "chill", "classical", "club", "comedy", "country",
                            #"dance", "dancehall", "death-metal", "deep-house", "detroit-techno", "disco", "disney", "drum-and-bass", "dub",
                            #"dubstep", "edm", "electro", "electronic", "emo", "folk", "forro", "french", "funk", "garage", "german", "gospel",
                            #"goth", "grindcore", "groove", "grunge", "guitar", "happy", "hard-rock", "hardcore", "hardstyle", "heavy-metal",
                            #"hip-hop", "holidays", "honky-tonk", "house", "idm", "indian", "indie", "indie-pop", "industrial", "iranian", "j-dance",
                            #"j-idol", "j-pop", "j-rock", "jazz", "k-pop", "kids", "latin", "latino", "malay", "mandopop", "metal", "metal-misc",
                            #"metalcore", "minimal-techno", "movies", "mpb", "new-age", "new-release", "opera", "pagode", "party", "philippines-opm",
                            #"piano", "pop", "pop-film", "post-dubstep", "power-pop", "progressive-house", "psych-rock", "punk", "punk-rock", "r-n-b",
                            #"rainy-day", "reggae", "reggaeton", "road-trip", "rock", "rock-n-roll", "rockabilly", "romance", "sad", "salsa", "samba",
                            #"sertanejo", "show-tunes", "singer-songwriter", "ska", "sleep", "songwriter", "soul", "soundtracks", "spanish", "study",
                            #"summer", "swedish", "synth-pop", "tango", "techno", "trance", "trip-hop", "turkish", "work-out", "world-music"
                            ]
        for genre in genres_to_remove:
            if genre in genres["genres"]:
                genres["genres"].remove(genre)
        print(genres)
    else:
        print("La richiesta HTTP ha restituito un codice di stato diverso da 200:", response.status_code, response.headers)

except Exception as e:
    print("\n", e)
    traceback.print_exc()

# Create a json file for genres list
with open("genres.json", 'w') as f:
   json.dump(genres, f, indent=4)

{
    "genres": [
        "acoustic",
        "afrobeat",
        "alt-rock",
        "alternative",
        "ambient",
        "anime",
        "black-metal",
        "bluegrass",
        "blues",
        "bossanova",
        "brazil",
        "breakbeat",
        "british",
        "cantopop",
        "chicago-house",
        "children",
        "chill",
        "classical",
        "club",
        "comedy",
        "country",
        "dance",
        "dancehall",
        "death-metal",
        "deep-house",
        "detroit-techno",
        "disco",
        "disney",
        "drum-and-bass",
        "dub",
        "dubstep",
        "edm",
        "electro",
        "electronic",
        "emo",
        "folk",
        "forro",
        "french",
        "funk",
        "garage",
        "german",
        "gospel",
        "goth",
        "grindcore",
        "groove",
        "grunge",
        "guitar",
        "happy",
        "hard-rock",
        "hardcore",
        "hardstyle",
  

In [113]:
min_number_of_tracks = 1000
max_null_iteration = 10
stored_tracks = {} # {track_id: genre}
old_tot_track = 0

for genre in tqdm(genres["genres"]):
    number_of_tracks = 0
    null_iteration_count = 0

    track_url = 'https://api.spotify.com/v1/recommendations?' + 'seed_genres=' + genre + "&limit=100"

    while number_of_tracks < min_number_of_tracks and null_iteration_count < max_null_iteration:
        try:
            if expires < datetime.datetime.now(): 
                print("Get new Token")
                expires, headers = get_new_token()
            # tracks_list = sp.recommendations(seed_genres=[genre], limit=100)

            response = requests.get(track_url, headers=headers)
            if response.status_code == 429:
                print(genre, len(stored_tracks)-old_tot_track)
                old_tot_track = len(stored_tracks)
                # Create a json file for genres list
                with open("stored_tracks.json", 'w') as f:
                    json.dump(stored_tracks, f, indent=4)
                print(response.headers)
                sys.exit()
            
            tracks_list = response.json()
        except Exception as e:
            print("\n", e)
            traceback.print_exc()
        
        null_iteration = True
        
        if "tracks" in tracks_list:
            for i, track in enumerate(tracks_list["tracks"]):
                if track["id"] not in stored_tracks:
                    stored_tracks[track["id"]] = genre
                    number_of_tracks += 1
                    null_iteration = False

        if null_iteration:
            null_iteration_count += 1
            
    print(genre, len(stored_tracks)-old_tot_track)
    old_tot_track = len(stored_tracks)
    # Create a json file for genres list
    with open("stored_tracks.json", 'w') as f:
       json.dump(stored_tracks, f, indent=4)
    
        

 50%|█████     | 1/2 [01:03<01:03, 63.83s/it]

work-out 847


100%|██████████| 2/2 [01:56<00:00, 58.24s/it]

world-music 849


In [6]:
# Create a json file for genres list
stored_tracks = "stored_tracks.json"

with open("stored_tracks.json", 'w') as f:
   json.dump(stored_tracks, f, indent=4)

NameError: name 'json' is not defined

In [7]:
columns = ["id", "track_name", "track_explicit",  "track_popularity", "album_name", "album_release_date", "album_release_date_precision",
           "artist_name", "audio_avg_pitches", "audio_avg_timbre",
           "audio_acousticness", "audio_danceability", "audio_duration_ms", "audio_energy", "audio_instrumentalness",
           "audio_key_1", "audio_liveness", "audio_loudness", "audio_mode_1", "audio_speechiness", "audio_tempo",
           "audio_time_signature", "audio_valence", "track_uri", "track_genre"]

with open('dataframe.csv', 'w', newline='') as f:
    write = csv.writer(f)
    write.writerow(columns)
    #write.writerows(list_of_track_audio_features)
    f.close()

NameError: name 'csv' is not defined

In [18]:
list_of_track_audio_features = []

with open('stored_tracks.json') as f:
    stored_tracks = json.load(f)

for i,track in enumerate(tqdm(stored_tracks)):

    if expires < datetime.datetime.now(): 
        print("Get new Token")
        expires, headers = get_new_token()
   
            
    try:
      track_info_url = "https://api.spotify.com/v1/tracks/" + track
      track_info = requests.get(track_info_url, headers=headers)
      track_audio_analysis_url = "https://api.spotify.com/v1/audio-analysis/" + track
      track_audio_analysis = requests.get(track_audio_analysis_url, headers=headers)
      features_url = "https://api.spotify.com/v1/audio-features/" + track
      features = requests.get(features_url, headers=headers)  

      valid_request_1 = track_info.status_code in range(200, 299)
      valid_request_2 = track_audio_analysis.status_code in range(200, 299)
      valid_request_3 = features.status_code in range(200, 299)

      if (valid_request_1 and valid_request_2 and valid_request_3):
        track_info = track_info.json()
        track_audio_analysis = track_audio_analysis.json()
        features = features.json()
      else:
        print("\n STATUS:", track_info.status_code, track_audio_analysis.status_code, features.status_code)
        continue
    except Exception as e:
      print(e)
      continue

    track_name = track_info["name"]     
    track_explicit = track_info['explicit']
    track_popularity = track_info["popularity"]
    album_info = track_info['album']
    album_name = album_info['name']
    album_release_date = album_info['release_date']
    album_release_date_precision = album_info['release_date_precision']
    artist_info = track_info['artists'][0]
    artist_name = artist_info['name']
    track_info = [track_name, track_explicit, track_popularity,
                  album_name, album_release_date, album_release_date_precision,
                  artist_name]
  
    #Add in some Spotify Audio Analysis Vectors
    num_seg = 0
    pitches = np.zeros(12)
    timbre = np.zeros(12)   
    if "segments" in track_audio_analysis:
        for _, j in enumerate(track_audio_analysis['segments']):
            pitches += np.array(j['pitches'])
            timbre += np.array(j['timbre'])
            num_seg+=1
    track_features_avg_pitches = list(pitches/num_seg)
    track_features_avg_timbre = list(timbre/num_seg)
    track_audio_analysis_list = [track_features_avg_pitches, track_features_avg_timbre]
  
    # Get audio features for this specific track
    features_list = [features['acousticness'], features['danceability'], features['duration_ms'],
                      features['energy'], features['instrumentalness'], features['key'], 
                      features['liveness'], features['loudness'], features['mode'],  
                      features['speechiness'], features['tempo'], features['time_signature'],
                      features['valence'], features['uri']]
    
    trackData = [track] + track_info + track_audio_analysis_list + features_list + [stored_tracks[track]]
    list_of_track_audio_features.append(trackData)

    with open('dataframe.csv', 'a+', newline='') as f:
    # using csv.writer method from CSV package
        write = csv.writer(f)
        write.writerow(trackData)
        f.close()

  5%|▌         | 1/19 [00:01<00:33,  1.87s/it]


 STATUS: 200 502 200


 11%|█         | 2/19 [00:04<00:34,  2.02s/it]


 STATUS: 200 502 200


 16%|█▌        | 3/19 [00:06<00:36,  2.29s/it]

...


 STATUS: 200 502 200


 95%|█████████▍| 18/19 [00:33<00:01,  1.81s/it]


 STATUS: 200 502 200


100%|██████████| 19/19 [00:35<00:00,  1.87s/it]


 STATUS: 200 502 200


In [14]:
spotify_track = pd.DataFrame(list_of_track_audio_features, columns = columns)
spotify_track.head()

NameError: name 'columns' is not defined